# Time Series Prediction with BQML and AutoML

**Objectives**
 1. Learn how to use BQML to create a classification time-series model using `CREATE MODEL`.
 2. Learn how to use BQML to create a linear regression time-series model.
 3. Learn how to use AutoML Tables to build a time series model from data in BigQuery.

## Set up environment variables and load necessary libraries

In [1]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
%env PROJECT = {PROJECT}
%env REGION = "us-central1" 

env: PROJECT=qwiklabs-gcp-00-eeb852ce8ccb
env: REGION="us-central1"


## Create the dataset

In [2]:
from google.cloud import bigquery
from IPython import get_ipython

bq = bigquery.Client(project=PROJECT)


def create_dataset():
    dataset = bigquery.Dataset(bq.dataset("stock_market"))
    try:
        bq.create_dataset(dataset)  # Will fail if dataset already exists.
        print("Dataset created")
    except:
        print("Dataset already exists")


def create_features_table():
    error = None
    try:
        bq.query('''
        CREATE TABLE stock_market.eps_percent_change_sp500
        AS
        SELECT *
        FROM `stock_market.eps_percent_change_sp500`
        ''').to_dataframe()
    except Exception as e:
        error = str(e)
    if error is None:
        print('Table created')
    elif 'Already Exists' in error:
        print('Table already exists.')
    else:
        raise Exception('Table was not created.')

create_dataset()
create_features_table()

Dataset already exists
Table already exists.


## Review the dataset

In the previous lab we created the data we will use modeling and saved them as tables in BigQuery. Let's examine that table again to see that everything is as we expect. Then, we will build a model using BigQuery ML using this table.

In [3]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  stock_market.eps_percent_change_sp500
LIMIT
  10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  6.46rows/s]


,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction,consensus_EPS,reported_EPS,surprise
0,MMM,2008-10-21,60.15,60.04,58.19,-1.85,0.910726,0.891073,0.891073,0.988841,...,9796,-0.030813,-0.061013,0.030200,3M Co,Industrials,UP,1.38,1.42,2.90
1,MMM,2006-10-20,78.09,78.47,80.09,1.62,0.960877,0.931184,0.867975,0.973620,...,9293,0.020645,0.006152,0.014493,3M Co,Industrials,UP,1.12,1.17,4.46
2,MMM,2009-04-24,54.26,57.00,57.35,0.35,0.911754,0.853509,0.733860,0.951053,...,9923,0.006140,-0.010067,0.016207,3M Co,Industrials,UP,0.86,0.81,-5.81
3,MMM,2006-07-25,71.00,68.11,69.06,0.95,1.038320,1.037439,1.028777,1.052562,...,9231,0.013948,-0.000378,0.014326,3M Co,Industrials,UP,1.07,1.05,-1.87
4,MMM,2004-07-19,85.50,83.05,85.00,1.95,1.052137,1.048645,0.831668,1.067068,...,8722,0.023480,0.007058,0.016422,3M Co,Industrials,UP,0.96,0.97,1.04
5,MMM,2007-04-26,80.31,80.45,81.55,1.10,0.956743,0.945805,0.846613,0.966439,...,9420,0.013673,-0.000120,0.013794,3M Co,Industrials,UP,1.12,1.28,14.29
6,MMM,2007-07-26,89.62,90.05,90.05,0.00,0.995225,0.963798,0.756358,1.012438,...,9483,0.000000,-0.015992,0.015992,3M Co,Industrials,UP,1.18,1.23,4.24
7,MMM,2005-01-18,83.99,82.02,82.17,0.15,1.021580,0.982931,0.912826,1.035845,...,8849,0.001829,-0.009490,0.011319,3M Co,Industrials,UP,0.91,0.91,0.00
8,CA,2012-01-24,22.42,22.82,25.03,2.21,0.946976,0.877739,0.815951,0.992550,...,6900,0.096845,0.008679,0.088166,CA Inc,Information Technology,UP,0.54,0.65,20.37
9,CA,2006-08-14,21.71,21.77,23.24,1.47,0.987598,0.877354,0.877354,1.005053,...,5529,0.067524,0.013696,0.053828,CA Inc,Information Technology,UP,0.13,0.17,30.77


## Using BQML

### Create classification model for `direction`

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

Have a look at [Step Two of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) to see another example.

**The query will take about two minutes to complete**


We'll start with creating a classification model to predict the `direction` of each stock. 

We'll take a random split using the `symbol` value. With about 500 different values, using `ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1` will give 30 distinct `symbol` values which corresponds to about 171,000 training examples. After taking 70% for training, we will be building a model on about 110,000 training examples.

In [4]:
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.direction_model OPTIONS(model_type = "logistic_reg",
    input_label_cols = ["direction"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  direction
FROM
  `stock_market.eps_percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 70

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1177.84query/s]                        


""


## Get training statistics and examine training info

After creating our model, we can evaluate the performance using the [`ML.EVALUATE` function](https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality#step_four_evaluate_your_model). With this command, we can find the precision, recall, accuracy F1-score and AUC of our classification model.

In [5]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.direction_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      direction
    FROM
      `stock_market.eps_percent_change_sp500`
    WHERE
      tomorrow_close IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) > 15 * 70
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 85))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.08s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.373016,0.376404,0.417178,0.366026,1.118389,0.513201


We can also examine the training statistics collected by Big Query. To view training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

In [6]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.direction_model`)
ORDER BY iteration

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.63rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.323281,0.360413,0.2,6169
1,0,1,0.258986,0.355889,0.4,10839


### Compare to simple benchmark

Another way to asses the performance of our model is to compare with a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicted the majority class. For the training dataset, the majority class is 'STAY'. The following query we can see how this naive strategy would perform on the eval set.

In [7]:
%%bigquery --project $PROJECT
#standardSQL
WITH
  eval_data AS (
  SELECT
    symbol,
    Date,
    Open,
    close_MIN_prior_5_days,
    close_MIN_prior_20_days,
    close_MIN_prior_260_days,
    close_MAX_prior_5_days,
    close_MAX_prior_20_days,
    close_MAX_prior_260_days,
    close_AVG_prior_5_days,
    close_AVG_prior_20_days,
    close_AVG_prior_260_days,
    close_STDDEV_prior_5_days,
    close_STDDEV_prior_20_days,
    close_STDDEV_prior_260_days,
    direction
  FROM
    `stock_market.eps_percent_change_sp500`
  WHERE
    tomorrow_close IS NOT NULL
    AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
    AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) > 15 * 70
    AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 85)
SELECT
  direction,
  (COUNT(direction)* 100 / (
    SELECT
      COUNT(*)
    FROM
      eval_data)) AS percentage
FROM
  eval_data
GROUP BY
  direction

Downloading: 100%|██████████| 3/3 [00:00<00:00,  3.55rows/s]


,direction,percentage
0,UP,28.834356
1,DOWN,27.607362
2,STAY,43.558282


So, the naive strategy of just guessing the majority class would have accuracy of 0.5509 on the eval dataset, just below our BQML model. 

### Create regression model for `normalized change`

We can also use BigQuery to train a regression model to predict the normalized change for each stock. To do this in BigQuery we need only change the OPTIONS when calling `CREATE OR REPLACE MODEL`. This will give us a more precise prediction rather than just predicting if the stock will go up, down, or stay the same. Thus, we can treat this problem as either a regression problem or a classification problem, depending on the business needs.


In [9]:
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.price_model OPTIONS(model_type = "linear_reg",
    input_label_cols = ["normalized_change"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  normalized_change
FROM
  `stock_market.eps_percent_change_sp500`
WHERE
  normalized_change IS NOT NULL
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 70

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1157.90query/s]                        


""


Just as before we can examine the evaluation metrics for our regression model and examine the training statistics in Big Query

In [10]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.price_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      normalized_change
    FROM
      `stock_market.eps_percent_change_sp500`
    WHERE
      normalized_change IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) > 15 * 70
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 85))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.04s/rows]


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.017972,0.000887,0.000958,0.011666,-0.115487,-0.104581


In [11]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.price_model`)
ORDER BY iteration

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.21rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.003059,0.0028,0.2,2709


## Train a Time Series model using AutoML Tables

### Step 1. Launch AutoML

Within the GCP console, navigate to Tables in the console menu.

<img src='../assets/console_menu_tables.png' width='50%'>

Click **Enable API**, if API is not enabled.

Click **GET STARTED**.


### Step 2. Create a Dataset

Select **New Dataset** and give it a name like `stock_market` and click *Create Dataset*. In the section on Importing data, select the option to import your data from a BigQuery Table. Fill in the details for your project, the dataset ID, and the table ID.

<img src='../assets/import_data_options.png' width='50%'>

### Step 3. Import the Data

Once you have created the dataset you can then import the data. This will take a few minutes.

<img src='../assets/importing_data.png' width='50%'>

### Step 4. Train the model 

Once the data has been imported into the dataset. You can examine the Schema of your data, Analyze the properties and values of the features and ultimately Train the model. Here you can also determine the label column and features for training the model. Since we are doing a classifcation model, we'll use `direction` as our target column.

<img src='../assets/schema_analyze_train.png' width='80%'>

Under the `Train` tab, click **Train Model**. You can choose the features to use when training. Select the same features as we used above.

<img src='../assets/train_model.png' width='50%'>

### Step 5. Evaluate your model.

Training can take many hours. But once training is complete you can inspect the evaluation metrics of your model. Since this is a classification task, we can also adjust the threshold and explore how different thresholds will affect your evaluation metrics. Also on that page, we can explore the feature importance of the various features used in the model and view confusion matrix for our model predictions.

<img src='../assets/eval_metrics.png' width='80%'>

### Step 6. Predict with the trained model. 

Once the model is done training, navigate to the Models page and Deploy the model, so we can test prediction. 

<img src='../assets/deploy_model.png' width='80%'>

When calling predictions, you can call batch prediction jobs by specifying a BigQuery table or csv file. Or you can do online prediction for a single instance.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License